# 1. Calculer la probabilité de $(S_n-pn)>\sqrt{n}$ où $S_n\leadsto \mathcal{B}(n, p)$ et comparer avec l'inégalité de Bienaymé-Tchebychev

## Rappel sur l'inégalité de Bienaymé-Tchebychev

**Inégalité de Bienaymé-Tchebychev**

Soit $X$ une variable aléatoire réelle et $\delta$ un réel.

Alors $\mathbb{P}\left(\vert X-E(X)\vert\geq \delta\right)\leq \dfrac{V(X)}{\delta^2}$.

*Remarques*

Cette inégalité donne une majoration de la probabilité que l'écart entre $X$ et son espérance dépasse une certaine valeur réelle $\delta$.

Ici on va utiliser l'inégalité de Bienaymé-Tchebichev avec $X=S_n$ ou $S_n\leadsto \mathcal{B}(n, p)$.

On sait que $E(S_n)=np$ et $V(S_n)=np(1-p)$.

On en déduit :

$\mathbb{P}\left(\vert S_n-np\vert\geq \delta\right)\leq \dfrac{np(1-p)}{\delta^2}$ pour tout réel $\delta$.

En choisissant $\delta=\sqrt{n}$, on obtient :

$\mathbb{P}\left(\vert S_n-np\vert\geq \sqrt{n}\right)\leq \dfrac{np(1-p)}{n}$

$\mathbb{P}\left(\vert S_n-np\vert\geq \sqrt{n}\right)\leq p(1-p)$

On va donc calculer $\mathbb{P}\left(\vert S_n-np\vert\geq \sqrt{n}\right)$ et comparer avec $p(1-p)$.

**Rappel**

Pour tout entier $k\in[0,n]$, $\mathbb{P}(S_n=k)=\dbinom{n}{k}p^k(1-p)^{n-k}$.

Il faut donc calculer $\dbinom{n}{k}$ pour tout entier $k\in[0,n]$.

Nous allons utiliser une méthode vue en NSI.

In [1]:
from math import sqrt

# L'idée est de remplir le triangle de Pascal en mémorisant les résultats au fur et à mesure
# et en utilisant la formule de Pascal
def coefbin(n, k):
    T = [[0] * (n + 1) for _ in range(n + 1)] # tableau de taille (n+1)*(n+1) ne contenant que des 0
    # les cases vides du triangle de Pascal seront donc ici des cases contenant 0
    for i in range(n+1): # remplissage par ligne
        for j in range(i + 1):
            if j == 0:
                T[i][j] = 1 # 1er nombre de la ligne
            else:
                T[i][j] = T[i - 1][j - 1] + T[i - 1][j]
                    # fonctionne même lorsque j == i car T[i - 1][i] vaut 0
    return T[n][k]

def binomFdp(n, p, k):
    return coefbin(n, k)*p**k*(1-p)**(n-k)

# Pour calculer P(|S_n-np|>sqrt(n)), on va 
# ajouter les probabilités des événements (|k-np|>sqrt(n)) pour k entier entre 0 et n
def BT(n, p):
    return sum([binomFdp(n, p, k) for k in range(n+1) if abs(k-n*p)>sqrt(n)])

# Valeurs à choisir
n = int(input("Quelle valeur de n voulez-vous tester ?\n"))
p = float(input("Quelle valeur de p voulez-vous tester ?\n"))
print(f"P(|S_n-np|>sqrt(n)) donne {BT(n, p)}.")
print(f"p(1-p) donne {p*(1-p)}.")

Quelle valeur de n voulez-vous tester ?
50
Quelle valeur de p voulez-vous tester ?
0.3
P(|S_n-np|>sqrt(n)) donne 0.019540328780510655.
p(1-p) donne 0.21.


L'inégalité est largement vérifiée. L'inégalité de BT est loin d'être optimale.

# 2. Simulation d'une marche aléatoire

On simule avec le module Turtle une **marche aléatoire** de $n$ pas ayant une même probabilité d´être effectués vers l’avant ou vers l’arrière. 

Pour que l’on voit la progression de Turtle le pas est réglé à 20 pixels et l’on change de couleur à chaque pas.

Dans graphe2, on simule $p$ déplacements de $n$ pas dans $360/p$ directions. 

Dans graphe3, on change d’orientation à chaque pas. 

Dans graphe4, on simule la marche aléatoire unidimensionnelle symétrique.

In [35]:
from random import *
from turtle import *
def un_saut(x):
    if random()<0.5:
        return x-1
    else:
        return x+1
def position(n):
    pos=0
    for i in range(n):
        pos += un_saut(0)
    return pos
def graphe1(n):
    colormode(255)
    goto(0,0)
    for i in range(n):
        forward(un_saut(0)*20) #Pas de 20 pixels
        color((randrange(0,256), randrange(0,256),randrange(0,256)))
def graphe2(p,n):
    for i in range(p):
        goto(0,0)
        setheading(0)
        right(180*i/p)
        graphe1(n)
def graphe3(n):
    reset()
    goto(0,0)
    for i in range(n):
        forward(un_saut(0)*20)
        right(90*(-1)**randrange(0,2))
def graphe4(n):
    reset()
    goto(0,0)
    setheading(0)
    for i in range(n):
        setheading(45*(-1)**randrange(0,2))
        forward(20)

# Choix de la marche
num = int(input("Quelle marche souhaitez-vous afficher ?\n"))
if num == 1:
    graphe1(20)
elif num == 2:
    graphe2(20,20)
elif num == 3:
    graphe3(20)
else:
    graphe4(20)
mainloop()

Quelle marche souhaitez-vous afficher ?
2


# 3. Simuler N échantillons de taille $n$ d'une variable aléatoire d'espérance $\mu$ et d'écart type $\sigma$. Calculer l'écart type s de la série des moyennes des échantillons observés, à comparer à $\dfrac{\sigma}{\sqrt{n}}$. Calculer la proportion des échantillons pour lesquels l'écart entre la moyenne et $\mu$ est inférieur ou égal à $ks$, ou à $\dfrac{k\sigma}{\sqrt{n}}$, pour $k = 1, 2, 3$.

Pour simuler N échantillons de taille $n$ d'une variable aléatoire d'espérance $\mu$ et d'écart-type $\sigma$, le choix fait ici est de considérer la loi normale (hors programme en Terminale) d'espérance $\mu$ et d'écart-type $\sigma$.

In [37]:
from random import normalvariate # import de la loi normale N(mu, sigma)

def simule_loi_normale(n, mu, sigma):
    L = [] # liste qui va contenir l'échantillon de taille n
    for k in range(n):
        L.append(normalvariate(mu, sigma))
    return L

On va maintenant programmer la série des moyennes de N échantillons de taille, puis en calculer l'écart-type $s$ et comparer avec $\dfrac{\sigma}{\sqrt{n}}$.

In [40]:
from math import sqrt

def moyenne(L): # calcul de la moyenne d'une liste L
    s = 0
    for k in L:
        s = s+k
    return s/len(L)

def ecart_type(L): # calcul de l'écart-type d'une liste L
    somme_carres_ecarts = 0
    m = moyenne(L)
    for val in L:
        somme_carres_ecarts = somme_carres_ecarts + (val - m)**2
    return sqrt(somme_carres_ecarts / len(L))

def serie_moyennes(N,n,mu,sigma):
    S = []
    for k in range(N):
        L = simule_loi_normale(n,mu,sigma)
        S.append(moyenne(L))
    return S

# Valeurs à modifier si besoin
N = 100
n = 50
mu = 10
sigma = 2
print(f"L'écart-type des {N} moyennes des échantillons de taille {n} de la loi normale N({mu}, {sigma}) est : {ecart_type(serie_moyennes(n, n, mu, sigma))}.")
print(f"En comparaison, sigma/sqrt(n) vaut {sigma/sqrt(n)}.")

L'écart-type des 100 moyennes des échantillons de taille 50 de la loi normale N(10, 2) est : 0.2568042404745911.
En comparaison, sigma/sqrt(n) vaut 0.282842712474619.


On va maintenant programmer une fonction donnant la proportion des échantillons pour lesquels l'écart entre la moyenne et $\mu$ est inférieur ou égal à $ks$ pour $k = 1, 2, 3$.

In [41]:
def prop_int_ks(N, n, mu, sigma, k):
    e = 0
    S = serie_moyennes(N, n, mu, sigma)
    s = ecart_type(S)
    for m in S:
        if abs(m - mu) < k * s:
            e = e + 1
    return e / N

# Valeurs à modifier si besoin
N = 100
n = 50
mu = 10
sigma = 2
k = 1
print(f"""La proportion des échantillons parmi les {N} échantillons de taille {n} de la loi normale
N({mu}, {sigma}) pour lesquels l'écart entre la moyenne et mu est inférieur ou égal à ks pour k = {k}
est {prop_int_ks(N, n, mu, sigma, k)}.""")

La proportion des échantillons parmi les 100 échantillons de taille 50 de la loi normale
N(10, 2) pour lesquels l'écart entre la moyenne et mu est inférieur ou égal à ks pour k = 1
est 0.65.


On va maintenant programmer une fonction donnant la proportion des échantillons pour lesquels l'écart entre la moyenne et $\mu$ est inférieur ou égal à $k*\sigma/\sqrt{n}$ pour $k = 1, 2, 3$.

In [42]:
def prop_int_ksigma(N, n, mu, sigma, k):
    e = 0
    S = serie_moyennes(N,n,mu,sigma)
    for m in S:
        if abs(m - mu) < k * sigma / sqrt(n):
            e = e + 1
    return e / N
# Valeurs à modifier si besoin
N = 100
n = 50
mu = 10
sigma = 2
k = 1
print(f"""La proportion des échantillons parmi les {N} échantillons de taille {n} de la loi normale
N({mu}, {sigma}) pour lesquels l'écart entre la moyenne et mu est inférieur ou égal à k*sigma/sqrt(n) 
pour k = {k} est {prop_int_ksigma(N, n, mu, sigma, k)}.""")

La proportion des échantillons parmi les 100 échantillons de taille 50 de la loi normale
N(10, 2) pour lesquels l'écart entre la moyenne et mu est inférieur ou égal à k*sigma/sqrt(n) 
pour k = 1 est 0.71.
